# Loading data with TimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import TimeBasedConfig # Time based dataset MUST use TimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
time_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.TIME_BASED, display_details=True)

[2025-08-26 09:20:00,599][wrapper_dataset][INFO] - Dataset is time-based. Use cesnet_tszoo.configs.TimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(ts_ids, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(ts_ids, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:00,604][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:00,623][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:00,627][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 483.84it/s]
[2025-08-26 09:20:00,744][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [298 241   6 406  34 ... 219 193 169  29  11], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
time_based_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")
# Or
time_based_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")

[2025-08-26 09:20:00,751][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:20:00,751][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:20:00,752][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:20:00,752][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:20:00,753][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
time_based_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
# Or
time_based_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)

[2025-08-26 09:20:00,758][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:20:00,758][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:20:00,759][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:20:00,759][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:20:00,759][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:20:00,767][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 212.91it/s]


(54, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = time_based_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:20:01,271][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 139.97it/s]


(54, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = time_based_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:20:01,511][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 105.09it/s]


(54, 128, 20)

#### All set

- Affected by `all_batch_size`.
- Affected by `all_workers`.

In [10]:
dataloader = time_based_dataset.get_all_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:20:01,626][cesnet_dataset][INFO] - Created new cached all_dataloader.
100%|██████████| 53/53 [00:00<00:00, 188.36it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [11]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:01,915][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:01,939][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:01,942][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 853.60it/s]
[2025-08-26 09:20:02,008][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [ 26 347 495 250 338 ...  42  39 161 140 191], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
   

In [12]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-26 09:20:02,016][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 227.89it/s]


(54, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [13]:
config = TimeBasedConfig(ts_ids=[177, 176, 319, 267], train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:02,488][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:02,507][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:02,510][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 4/4 [00:00<00:00, 533.17it/s]
[2025-08-26 09:20:02,519][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [177 176 319 267], Length=4
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Slidin

In [14]:
dataloader = time_based_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:20:02,527][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 1061.08it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(ts_ids, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(ts_ids, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [15]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:02,634][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:02,652][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:02,655][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 2248.40it/s]
[2025-08-26 09:20:02,681][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [545 446 287 228 505 ... 539 300 238 378 466], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 335
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
        Anomaly handler type: None        
    Batch sizes
        Train batch size: 32
        Val batch size: 64

In [16]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-26 09:20:02,690][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 5010.65it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [17]:
time_based_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
time_based_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-26 09:20:02,795][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:20:02,796][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-26 09:20:02,796][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:20:02,797][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:20:02,797][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:20:02,798][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [18]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:02,806][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:02,826][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:02,829][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 2115.86it/s]
[2025-08-26 09:20:02,855][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [167 157 222 302 112 ... 181  38 469 304 281], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 100
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
        Anomaly handler type: None        
    Batch sizes
        Train batch size: 32
        Val batch size: 6

In [19]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-26 09:20:02,865][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 5394.70it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [20]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:02,963][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:02,982][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:02,985][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 654.01it/s]
[2025-08-26 09:20:03,070][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [253 485 358 452 233 ... 158  60 538  63 387], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

#### Train set

- Affected by `train_workers`.

In [21]:
df = time_based_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,253.0,0.0,3993.0,12262.0,4.819489e+06,3335.0,4.87,2.14,3662.0,5.35,2.49,3705.0,5.41,2.640000,0.950195,0.930176,0.409912,0.409912,0.83,141.399994
1,253.0,1.0,4333.0,10780.0,3.196162e+06,3564.0,5.09,2.48,4008.0,5.73,3.14,3996.0,5.71,3.070000,0.930176,0.910156,0.399902,0.389893,0.75,136.699997
2,253.0,2.0,5194.0,76853.0,6.782261e+07,3361.0,4.92,2.55,3647.0,5.34,2.66,3867.0,5.66,4.400000,0.930176,0.910156,0.419922,0.409912,0.88,134.429993
3,253.0,3.0,6587.0,649647.0,7.588314e+08,3066.0,4.65,4.29,3088.0,4.68,2.82,4403.0,6.67,21.120001,0.939941,0.919922,0.459961,0.439941,1.28,129.429993
4,253.0,4.0,8369.0,544770.0,4.646289e+08,2565.0,4.04,5.28,2397.0,3.77,2.37,4747.0,7.48,31.049999,0.939941,0.919922,0.479980,0.469971,0.75,125.849998
5,253.0,5.0,9365.0,1232213.0,1.244550e+09,2460.0,4.05,6.51,2143.0,3.53,2.38,5139.0,8.47,36.139999,0.950195,0.930176,0.489990,0.479980,1.21,128.309998
6,253.0,6.0,9451.0,1247653.0,1.334478e+09,2496.0,4.19,6.41,2221.0,3.73,2.40,5186.0,8.70,39.020000,0.939941,0.919922,0.469971,0.459961,1.04,131.380005
7,253.0,7.0,8566.0,743959.0,7.116519e+08,2578.0,4.17,5.88,2440.0,3.95,2.42,4899.0,7.93,33.709999,0.930176,0.919922,0.459961,0.449951,1.52,130.350006
8,253.0,8.0,8607.0,947064.0,9.099148e+08,2568.0,4.14,6.18,2267.0,3.65,2.18,5066.0,8.16,35.410000,0.930176,0.899902,0.479980,0.459961,0.94,123.669998
9,253.0,9.0,7792.0,425931.0,3.531234e+08,2469.0,4.07,5.25,2290.0,3.78,2.48,4718.0,7.79,30.219999,0.939941,0.919922,0.469971,0.459961,1.04,129.100006


In [22]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     253.0      0.0   3993.0    12262.0    4819489.0   
 1                     253.0      1.0   4333.0    10780.0    3196162.0   
 2                     253.0      2.0   5194.0    76853.0   67822613.0   
 3                     253.0      3.0   6587.0   649647.0  758831416.0   
 4                     253.0      4.0   8369.0   544770.0  464628877.0   
 ...                     ...      ...      ...        ...          ...   
 3354                  253.0   3354.0   3249.0    28933.0   28148353.0   
 3355                  253.0   3355.0   5134.0    19937.0   15232316.0   
 3356                  253.0   3356.0   4372.0    33024.0   33444108.0   
 3357                  253.0   3357.0   2628.0    22938.0   21058195.0   
 3358                  253.0   3358.0   2585.0    36969.0   40334003.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             3335.0            4.

#### Val set

- Affected by `val_workers`.

In [23]:
df = time_based_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,253.0,3359.0,3203.0,46648.0,49390034.0,2079.0,3.46,2.15,2256.0,3.76,2.48,2424.0,4.04,3.91,0.919922,0.910156,0.520020,0.500000,0.36,116.430000
1,253.0,3360.0,2872.0,24399.0,24437311.0,2020.0,3.36,2.31,2180.0,3.62,3.15,2294.0,3.81,3.55,0.910156,0.879883,0.529785,0.500000,0.93,115.050003
2,253.0,3361.0,3104.0,42458.0,45404546.0,2051.0,3.41,2.09,2189.0,3.64,2.79,2368.0,3.93,3.85,0.919922,0.899902,0.509766,0.489990,0.41,119.779999
3,253.0,3362.0,2924.0,14442.0,9732587.0,1906.0,3.23,1.88,1981.0,3.36,2.14,2198.0,3.73,3.53,0.910156,0.890137,0.500000,0.479980,0.53,117.910004
4,253.0,3363.0,3054.0,43204.0,46174143.0,1926.0,3.20,2.06,2074.0,3.45,2.51,2244.0,3.73,4.06,0.930176,0.910156,0.509766,0.500000,0.47,123.330002
5,253.0,3364.0,2346.0,27769.0,28739001.0,1448.0,2.84,1.78,1555.0,3.06,2.07,1734.0,3.41,3.78,0.919922,0.890137,0.520020,0.500000,0.34,120.269997
6,253.0,3365.0,4850.0,224332.0,253529953.0,1360.0,2.68,2.23,1336.0,2.63,1.68,1715.0,3.38,6.50,0.950195,0.939941,0.569824,0.560059,0.57,115.519997
7,253.0,3366.0,2718.0,93350.0,92512824.0,1380.0,2.86,2.62,1346.0,2.79,2.15,1713.0,3.55,6.21,0.939941,0.919922,0.569824,0.549805,0.70,117.620003
8,253.0,3367.0,2395.0,37363.0,29496848.0,1345.0,2.85,2.79,1295.0,2.74,1.99,1664.0,3.53,6.90,0.939941,0.930176,0.569824,0.549805,1.61,113.489998
9,253.0,3368.0,2450.0,13678.0,4246860.0,1411.0,2.74,2.04,1503.0,2.92,2.34,1633.0,3.17,3.70,0.930176,0.910156,0.549805,0.529785,0.85,120.750000


In [24]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     253.0   3359.0   3203.0    46648.0  49390034.0   
 1                     253.0   3360.0   2872.0    24399.0  24437311.0   
 2                     253.0   3361.0   3104.0    42458.0  45404546.0   
 3                     253.0   3362.0   2924.0    14442.0   9732587.0   
 4                     253.0   3363.0   3054.0    43204.0  46174143.0   
 ...                     ...      ...      ...        ...         ...   
 2010                  253.0   5369.0   1834.0    10094.0   7136949.0   
 2011                  253.0   5370.0   2007.0    12755.0   7561361.0   
 2012                  253.0   5371.0   1807.0     7544.0   3289592.0   
 2013                  253.0   5372.0   2230.0    21376.0  15894554.0   
 2014                  253.0   5373.0   1988.0     7154.0   3413343.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             2079.0            3.46          

#### Test set

- Affected by `test_workers`.

In [25]:
df = time_based_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,253.0,5374.0,1967.0,4887.0,8.015970e+05,1421.0,2.88,1.92,1811.0,3.67,5.50,1642.0,3.32,2.540000,0.910156,0.890137,0.479980,0.459961,1.33,124.169998
1,253.0,5375.0,2209.0,5324.0,6.019300e+05,1518.0,3.06,2.18,2042.0,4.12,7.07,1780.0,3.59,2.840000,0.919922,0.890137,0.469971,0.439941,1.20,123.970001
2,253.0,5376.0,1979.0,4461.0,4.574890e+05,1408.0,2.97,2.03,1839.0,3.88,6.11,1643.0,3.47,2.670000,0.939941,0.919922,0.459961,0.439941,1.83,127.800003
3,253.0,5377.0,2031.0,4699.0,7.499130e+05,1466.0,3.02,1.85,1842.0,3.80,5.76,1697.0,3.50,2.580000,0.930176,0.899902,0.459961,0.429932,1.54,131.190002
4,253.0,5378.0,2755.0,81613.0,9.308068e+07,1267.0,2.92,2.59,1583.0,3.65,5.64,1572.0,3.62,5.750000,0.899902,0.879883,0.489990,0.459961,1.17,126.839996
5,253.0,5379.0,10816.0,1341129.0,1.617607e+09,1442.0,3.29,4.67,1575.0,3.60,5.38,3128.0,7.14,28.480000,0.910156,0.890137,0.500000,0.479980,1.27,122.879997
6,253.0,5380.0,6523.0,442422.0,4.075886e+08,1545.0,3.59,6.02,1572.0,3.66,5.13,3810.0,8.86,37.000000,0.919922,0.899902,0.509766,0.479980,1.63,124.260002
7,253.0,5381.0,7779.0,640961.0,6.382331e+08,1525.0,3.71,7.10,1387.0,3.37,4.28,4242.0,10.32,45.119999,0.899902,0.879883,0.509766,0.479980,2.22,122.559998
8,253.0,5382.0,6175.0,648685.0,6.671700e+08,1400.0,3.36,5.37,1431.0,3.43,4.66,3509.0,8.41,38.830002,0.919922,0.899902,0.520020,0.489990,1.47,126.160004
9,253.0,5383.0,6569.0,563136.0,5.229070e+08,1539.0,3.54,5.79,1584.0,3.64,5.62,3779.0,8.69,38.900002,0.930176,0.910156,0.520020,0.489990,1.82,122.709999


In [26]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     253.0   5374.0   1967.0     4887.0    801597.0   
 1                     253.0   5375.0   2209.0     5324.0    601930.0   
 2                     253.0   5376.0   1979.0     4461.0    457489.0   
 3                     253.0   5377.0   2031.0     4699.0    749913.0   
 4                     253.0   5378.0   2755.0    81613.0  93080679.0   
 ...                     ...      ...      ...        ...         ...   
 1338                  253.0   6712.0   2758.0    11547.0   3697882.0   
 1339                  253.0   6713.0   2459.0    10350.0   4117562.0   
 1340                  253.0   6714.0   2540.0     9778.0   1975445.0   
 1341                  253.0   6715.0   2940.0     7908.0    998295.0   
 1342                  253.0   6716.0   3289.0     8211.0   1131492.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             1421.0            2.88          

In [27]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     253.0   5374.0   1967.0     4887.0    801597.0   
 1                     253.0   5375.0   2209.0     5324.0    601930.0   
 2                     253.0   5376.0   1979.0     4461.0    457489.0   
 3                     253.0   5377.0   2031.0     4699.0    749913.0   
 4                     253.0   5378.0   2755.0    81613.0  93080679.0   
 ...                     ...      ...      ...        ...         ...   
 1338                  253.0   6712.0   2758.0    11547.0   3697882.0   
 1339                  253.0   6713.0   2459.0    10350.0   4117562.0   
 1340                  253.0   6714.0   2540.0     9778.0   1975445.0   
 1341                  253.0   6715.0   2940.0     7908.0    998295.0   
 1342                  253.0   6716.0   3289.0     8211.0   1131492.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             1421.0            2.88          

#### All set

- Affected by `all_workers`.

In [28]:
df = time_based_dataset.get_all_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_all_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,253.0,0.0,3993.0,12262.0,4.819489e+06,3335.0,4.87,2.14,3662.0,5.35,2.49,3705.0,5.41,2.640000,0.950195,0.930176,0.409912,0.409912,0.83,141.399994
1,253.0,1.0,4333.0,10780.0,3.196162e+06,3564.0,5.09,2.48,4008.0,5.73,3.14,3996.0,5.71,3.070000,0.930176,0.910156,0.399902,0.389893,0.75,136.699997
2,253.0,2.0,5194.0,76853.0,6.782261e+07,3361.0,4.92,2.55,3647.0,5.34,2.66,3867.0,5.66,4.400000,0.930176,0.910156,0.419922,0.409912,0.88,134.429993
3,253.0,3.0,6587.0,649647.0,7.588314e+08,3066.0,4.65,4.29,3088.0,4.68,2.82,4403.0,6.67,21.120001,0.939941,0.919922,0.459961,0.439941,1.28,129.429993
4,253.0,4.0,8369.0,544770.0,4.646289e+08,2565.0,4.04,5.28,2397.0,3.77,2.37,4747.0,7.48,31.049999,0.939941,0.919922,0.479980,0.469971,0.75,125.849998
5,253.0,5.0,9365.0,1232213.0,1.244550e+09,2460.0,4.05,6.51,2143.0,3.53,2.38,5139.0,8.47,36.139999,0.950195,0.930176,0.489990,0.479980,1.21,128.309998
6,253.0,6.0,9451.0,1247653.0,1.334478e+09,2496.0,4.19,6.41,2221.0,3.73,2.40,5186.0,8.70,39.020000,0.939941,0.919922,0.469971,0.459961,1.04,131.380005
7,253.0,7.0,8566.0,743959.0,7.116519e+08,2578.0,4.17,5.88,2440.0,3.95,2.42,4899.0,7.93,33.709999,0.930176,0.919922,0.459961,0.449951,1.52,130.350006
8,253.0,8.0,8607.0,947064.0,9.099148e+08,2568.0,4.14,6.18,2267.0,3.65,2.18,5066.0,8.16,35.410000,0.930176,0.899902,0.479980,0.459961,0.94,123.669998
9,253.0,9.0,7792.0,425931.0,3.531234e+08,2469.0,4.07,5.25,2290.0,3.78,2.48,4718.0,7.79,30.219999,0.939941,0.919922,0.469971,0.459961,1.04,129.100006


In [29]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     253.0      0.0   3993.0    12262.0    4819489.0   
 1                     253.0      1.0   4333.0    10780.0    3196162.0   
 2                     253.0      2.0   5194.0    76853.0   67822613.0   
 3                     253.0      3.0   6587.0   649647.0  758831416.0   
 4                     253.0      4.0   8369.0   544770.0  464628877.0   
 ...                     ...      ...      ...        ...          ...   
 6712                  253.0   6712.0   2758.0    11547.0    3697882.0   
 6713                  253.0   6713.0   2459.0    10350.0    4117562.0   
 6714                  253.0   6714.0   2540.0     9778.0    1975445.0   
 6715                  253.0   6715.0   2940.0     7908.0     998295.0   
 6716                  253.0   6716.0   3289.0     8211.0    1131492.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             3335.0            4.

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [30]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:04,695][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:04,715][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:04,718][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 692.04it/s]
[2025-08-26 09:20:04,798][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [365 418  86 422 476 ... 498 206 471  41  33], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

#### Train set

- Affected by `train_workers`.

In [31]:
numpy_array = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(54, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [32]:
numpy_array = time_based_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(54, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [33]:
numpy_array = time_based_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### All set

- Affected by `all_workers`.

In [34]:
numpy_array = time_based_dataset.get_all_numpy(workers="config")

display(numpy_array.shape)

(54, 6717, 20)

#### Using time_format=TimeFormat.DATETIME

In [35]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:20:05,127][time_config][INFO] - Quick validation succeeded.
[2025-08-26 09:20:05,149][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:20:05,154][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 1199.48it/s]
[2025-08-26 09:20:05,202][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [333 383 176 521 482 ... 496 274 389 373 441], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
   

In [36]:
numpy_array, times = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(54, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)